In [2]:
import json
import requests
import re
import psycopg2

In [34]:
db_cnx = psycopg2.connect(user="docker", password="docker", host="localhost", database="tcc")

In [5]:
detalhe_proposicoes_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/"
autores_proposicao_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/:id/autores"
codigo_situacao_url = "https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codSituacao"

## Buscando proposições

In [6]:
codigo_situacao_lista = requests.get(codigo_situacao_url).json()["dados"]
dicionario_cod_situacao = {}
for i in codigo_situacao_lista:
    dicionario_cod_situacao[i["cod"]] = i["nome"]

In [ ]:
proposicoes_json = "./proposicoes-2023.json"

In [7]:
with open(proposicoes_json) as f:
    proposicoes = json.load(f)["dados"]

query = """
INSERT INTO proposicoes (id, tipo, ano, ementa, ementa_detalhada, uri_documento, data_apresentacao, ultimo_relator_id, situacao)
VALUES (%(id)s, %(siglaTipo)s, %(ano)s, %(ementa)s, null, %(uri)s, null, null, null)
ON CONFLICT (id) DO UPDATE 
    SET tipo = %(siglaTipo)s,
        ano = %(ano)s,
        ementa = %(ementa)s,
        ementa_detalhada = null,
        uri_documento = null,
        data_apresentacao = null,
        ultimo_relator_id = null,
        situacao = null;
"""
cursor = db_cnx.cursor()
for proposicao in proposicoes:
    to_insert = {key: proposicao[key] for key in ['id', 'uri', 'siglaTipo', 'codTipo', 'numero', 'ano', 'ementa']}
    cursor.execute(query, to_insert)
db_cnx.commit()
cursor.close()

True

In [10]:
def extrair_id_deputado(url):
    return re.search(r"/deputados/(.*)$", url).group(1)

In [8]:
query = """
UPDATE proposicoes 
SET ementa_detalhada = %(ementaDetalhada)s, 
    data_apresentacao = %(dataApresentacao)s,
    uri_documento = %(urlInteiroTeor)s,
    ultimo_relator_id = %(ultimoRelatorId)s,
    situacao = %(situacao)s 
WHERE id = %(id)s;

"""

query_keywords = """
    insert into proposicoes_keywords(keyword, proposicao_id) values (%(keyword)s, %(id)s);
"""

cursor = db_cnx.cursor()
ids = []
for proposicao in proposicoes:
    params = {
        "ementaDetalhada": proposicao["ementaDetalhada"],
        "dataApresentacao": proposicao["dataApresentacao"][:-6],
        "id": proposicao['id'],
        "urlInteiroTeor": proposicao["urlInteiroTeor"]
    }
    uriUltimoRelator = proposicao["ultimoStatus"]["uriRelator"]
    ultimoRelatorId = None
    if not uriUltimoRelator is None and uriUltimoRelator != "":
        ultimoRelatorId = re.search(r"/deputados/(.*)$", uriUltimoRelator).group(1)
    cod_situacao = proposicao["ultimoStatus"]["idSituacao"]
    params["situacao"] = None
    if not cod_situacao is None and cod_situacao != "":
        params["situacao"] = dicionario_cod_situacao[str(cod_situacao)]
    params["ultimoRelatorId"] = ultimoRelatorId

    cursor.execute(query, params)

    cursor.execute("delete from proposicoes_keywords where proposicao_id = %(id)s", {"id": params["id"]})
    for i in proposicao["keywords"].split(", "):
        cursor.execute(query_keywords, {"keyword": unidecode(i.strip().lower().replace(".", "")), "id": params["id"]})
    ids.append(proposicao['id'])
db_cnx.commit()    

True

In [39]:
statement_autor_proposicao = """
                    insert ignore into autor_proposicoes (id_proposicao, id_deputado)
                    values (%s, %s);
                """

with open("proposicoesAutores-2023.json") as f:
    autores_proposicoes = json.load(f)["dados"]
    
deputados_nao_existe = []
    
for autor_prop in autores_proposicoes:
    if autor_prop.get("tipoAutor") != "Deputado":
        continue
    
    id_autor = extrair_id_deputado(autor_prop.get("uriAutor"))
    cursor.execute("select exists(select * from proposicoes where id = %s)", [autor_prop["idProposicao"]])
    proposicao_existe = cursor.fetchone()[0] == 1
    if not proposicao_existe:
        continue
        
    cursor.execute("select exists(select * from deputado where id = %s)", [id_autor])
    deputado_existe = cursor.fetchone()[0] == 1
    
    if not deputado_existe:
        deputados_nao_existe.append(id_autor)
        continue
    
    cursor.execute(statement_autor_proposicao, (autor_prop.get("idProposicao"), id_autor))
db_cnx.commit()

In [45]:
insert_deputado_statement = """
    insert ignore into deputado (id, nome, nome_eleitoral, situacao,
                          condicao_eleitoral, partido, estado, uri_foto)
    values ( %(id)s, %(nome)s, %(nome_eleitoral)s, %(situacao)s,
     %(condicao_eleitoral)s,
     %(partido)s,
     %(estado)s,
     %(uri_foto)s
     );"""
for deputado in deputados_nao_existe:
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado}"
    data = requests.get(url).json()["dados"]
    params = {
        "id": data["id"],
        "nome_eleitoral": data["ultimoStatus"]["nomeEleitoral"],
        "estado": data["ultimoStatus"]["siglaUf"],
        "partido": data["ultimoStatus"]["siglaPartido"],
        "uri_foto": data["ultimoStatus"]["urlFoto"],
        "nome": data["nomeCivil"],
        "situacao": data["ultimoStatus"]["situacao"],
        "condicao_eleitoral": data["ultimoStatus"]["condicaoEleitoral"]
    }
    cursor.execute(insert_deputado_statement, params)
db_cnx.commit()

In [46]:
## tentando dnv...

statement_autor_proposicao = """
                    insert ignore into autor_proposicoes (id_proposicao, id_deputado)
                    values (%s, %s);
                """

with open("proposicoesAutores-2023.json") as f:
    autores_proposicoes = json.load(f)["dados"]
    
deputados_nao_existe = []

for autor_prop in autores_proposicoes:
    if autor_prop.get("tipoAutor") != "Deputado":
        continue
    
    id_autor = extrair_id_deputado(autor_prop.get("uriAutor"))
    cursor.execute("select exists(select * from proposicoes where id = %s)", [autor_prop["idProposicao"]])
    proposicao_existe = cursor.fetchone()[0] == 1
    if not proposicao_existe:
        continue
        
    cursor.execute("select exists(select * from deputado where id = %s)", [id_autor])
    deputado_existe = cursor.fetchone()[0] == 1
    
    if not deputado_existe:
        deputados_nao_existe.append(id_autor)
        continue
    
    cursor.execute(statement_autor_proposicao, (autor_prop.get("idProposicao"), id_autor))
db_cnx.commit()

In [49]:
db_cnx.close()